In [1]:
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end

"/Users/allisonlettiere/Desktop/CS238FinalProject/Project.toml"

In [2]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using StaticArrays
import POMDPs.reward

In [21]:
sensor = Bumper()
config = 1# 1,2, or 3

num_x_pts = 50
num_y_pts = 50
num_th_pts = 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts)
aspace = vec(collect(RoombaAct(v, om) for v in [0,1,10], om in [-1,0,1]))

m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace))

RoombaPOMDP{Bumper,Bool}(Bumper(), RoombaMDP{DiscreteRoombaStateSpace,Array{RoombaAct,1}}
  v_max: Float64 10.0
  om_max: Float64 1.0
  dt: Float64 0.5
  contact_pen: Float64 -1.0
  time_pen: Float64 -0.1
  goal_reward: Float64 10.0
  stairs_penalty: Float64 -10.0
  config: Int64 1
  sspace: DiscreteRoombaStateSpace
  room: AA228FinalProject.Room
  aspace: Array{RoombaAct}((15,))
  _amap: Dict{RoombaAct,Int64}
)

In [22]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

mdp(e::RoombaMDP) = e
mdp(e::RoombaPOMDP) = e.mdp

mdp (generic function with 2 methods)

## Randomized Point Based Value Iteration Policy Building

In [24]:
discountFactor = 0.9
goal_x, goal_y = get_goal_xy(m)

if(config == 1 || config == 3)
    ang_to_goal = 0.0
end
if(config == 2)
    ang_to_goal = -2.0
end

si = RoombaState(-1.5, -1.5, 1.0, 0.)
sf = RoombaState(goal_x, goal_y, 1.0, 0.)

action = (10.0, ang_to_goal)
startingReward = rew(m, si, action, sf)
firstAlphaVector = []

for state in 1:length(POMDPs.states(m))
    push!(firstAlphaVector, (1/(1-discountFactor)) * startingReward)
end

In [27]:
B = []
dist = initialstate_distribution(m)
b = initialize_belief(belief_updater, dist)

for i = 1:100
    push!(B, b)
    b = resample(resampler, b, belief_updater.spf.rng)
end

gamma = Dict()
gamma[action] = firstAlphaVector
gamma = @time RandomizedPointBasedUpdate(B, gamma)

In [29]:
gamma2 = Dict()
gamma2[action] = firstAlphaVector

B = []
dist = initialstate_distribution(m)
b = initialize_belief(belief_updater, dist)

for i = 1:100
    push!(B, b)
    b = resample(resampler, b, belief_updater.spf.rng)
end

gamma2 = @time RandomizedPointBasedUpdate(B, gamma)

hellohellohello194.012022 seconds (2.46 G allocations: 159.132 GiB, 22.50% gc time)


Dict{Any,Any} with 15 entries:
  [5.0, 0.0]   => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [3.0, 2.0]   => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [10.0, 1.0]  => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [10.0, 2.0]  => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [5.0, 2.0]   => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [3.0, -2.0]  => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [3.0, -1.0]  => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [10.0, -1.0] => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [5.0, 1.0]   => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [5.0, -2.0]  => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [5.0, -1.0]  => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.…
  [10.0, 0.0]  => Any[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1

In [ ]:
gamma3 = Dict()
gamma3[action] = firstAlphaVector

B = []
dist = initialstate_distribution(m)
b = initialize_belief(belief_updater, dist)

for i = 1:100
    push!(B, b)
    b = resample(resampler, b, belief_updater.spf.rng)
end

gamma3 = @time RandomizedPointBasedUpdate(B, gamma2)

In [ ]:
B = []
dist = initialstate_distribution(m)
b = initialize_belief(belief_updater, dist)

for i = 1:100
    push!(B, b)
    b = resample(resampler, b, belief_updater.spf.rng)
end

gamma4 = Dict()
gamma4[action] = firstAlphaVector

gamma4 = @time RandomizedPointBasedUpdate(B, gamma3)

In [ ]:
B = []
dist = initialstate_distribution(m)
b = initialize_belief(belief_updater, dist)

for i = 1:100
    push!(B, b)
    b = resample(resampler, b, belief_updater.spf.rng)
end

gamma5 = Dict()
gamma5[action] = firstAlphaVector

gamma5 = @time RandomizedPointBasedUpdate(B, gamma4)

In [ ]:
gamma10 = Dict()
gamma10[action] = firstAlphaVector
for i = 1:10
    B = []
    dist = initialstate_distribution(m)
    b = initialize_belief(belief_updater, dist)

    for i = 1:100
        push!(B, b)
        b = resample(resampler, b, belief_updater.spf.rng)
    end
    gamma10 = @time RandomizedPointBasedUpdate(B, gamma10)
end

In [ ]:
gamma25 = Dict()
gamma25[action] = firstAlphaVector
for i = 1:25
    B = []
    dist = initialstate_distribution(m)
    b = initialize_belief(belief_updater, dist)

    for i = 1:100
        push!(B, b)
        b = resample(resampler, b, belief_updater.spf.rng)
    end
    gamma25 = @time RandomizedPointBasedUpdate(B, gamma25)
end

In [8]:
function probabilityVector(m, b)    
    probabilitiesB = zeros(length(POMDPs.states(m)))   
    ps = probdict(b)
    for i in keys(ps)
        if(i in (POMDPs.states(m)))
            location = findall(x->x==i, POMDPs.states(m))[1]
            probabilitiesB[location] = ps[i]
        end 
    end
    return probabilitiesB
end

probabilityVector (generic function with 1 method)

In [9]:
function probdict(b::AbstractParticleBelief{S}) where {S}
    probs = Dict{S, Float64}()
    for (i,p) in enumerate(particles(b))
        if haskey(probs, p)
            probs[p] += weight(b, i)/weight_sum(b)
        else
            probs[p] = weight(b, i)/weight_sum(b)
        end
    end
    return probs
end

probdict (generic function with 1 method)

In [10]:
function RandomizedPointBasedUpdate(B, gamma)
    BP = B
    gammaP = Dict()
    while !(isempty(BP))
        b = rand(BP)
        filter!(e->e != b,BP)
        alphaVectors = backupBelief(b, gamma)
        probabilities = probabilityVector(m, b)
        for action in aspace
            if (action in keys(gammaP))
                maxU = -1000
                oldMaxAlpha = gammaP[action] 
                oldUtility = oldMaxAlpha'probabilities
                newUtility = alphaVectors[action]'probabilities
                if (newUtility >= oldUtility)
                    gammaP[action] = alphaVectors[action]
                else
                    gammaP[action] = oldMaxAlpha
                end
            else
                gammaP[action] = alphaVectors[action]
            end
        end  
    end
    return gammaP
end

RandomizedPointBasedUpdate (generic function with 1 method)

In [11]:
function backupBelief(b, gamma)
    O = [0, 1]
    alphaAO = Dict()
    alphaVectors = Dict()
    actionNumber = 1

    for action in aspace
        alphaStateAction = []
        for observation in O
            alphaAO[[action, observation + 1]] = zeros(length(POMDPs.states(m)))
            bp = AA228FinalProject.update(belief_updater, b, action, observation)
            maxTranspose = -1000
            probability = probabilityVector(m, bp)
            for a in keys(gamma)
                t = gamma[a]'probability
                if (t > maxTranspose)
                    maxTranspose = t
                    alphaAO[[action, observation + 1]] = gamma[a]
                end
            end
        end 
        
        s = mean(b)
        e, gr, gwn, sr, swn, next_x, next_y, next_th, next_status = collisionStatus(s, action)
        sp = RoombaState(next_x, next_y, next_th, next_status)

        alpha = Dict()        
        stateNumber = 1
        for state in POMDPs.states(m)  
            push!(alphaStateAction, AA228FinalProject.reward(m, s, action, sp) + discountFactor * sumNextStateObservation(s, action, alphaAO, stateNumber))
            stateNumber += 1
        end
    
        alphaVectors[action] = alphaStateAction
        actionNumber += 1
    end
    return alphaVectors
end     

backupBelief (generic function with 1 method)

In [12]:
function sumNextStateObservation(s, a, alphaAO, stateNumber)
    sum = 0
    e, gr, gwn, sr, swn, next_x, next_y, next_th, next_status, sp = collisionStatus(s, a)
    
    ##0 -- does not hit wall (false), 1 -- hits wall (true)
    for o in [0, 1]
        observationFunction = 0
        if o == 0
            if(!wall_contact(e, s))
                observationFunction = 1
                ob = 0
            end
        elseif o == 1
            if(wall_contact(e, s))
                observationFunction = 1
            end
        end
        transitionFunction = 1 
        sum += observationFunction * transitionFunction * alphaAO[[a, o + 1]][stateNumber]
    end
    return sum
end

sumNextStateObservation (generic function with 1 method)

In [34]:
mutable struct ToEnd <: Policy
    ts::Int64 # to track the current time-step.
end

function POMDPs.action(p::ToEnd, b::ParticleCollection{RoombaState})
    maxUtility = -1000
    actionSelection = 0
    probability = probabilityVector(m, b)
    for action in keys(gamma2)
        alpha = gamma[action]
        transpose = alpha'probability
        if (transpose > maxUtility)
            maxUtility = transpose
            actionSelection = action
        end
    end
    return RoombaAct(actionSelection[1], actionSelection[2])
end

## Create and Run the Environment

In [35]:
# first seed the environment
Random.seed!(2)

# reset the policy
p = ToEnd(0) # here, the argument sets the time-steps elapsed to 0

# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget

        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)

        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

In [36]:
using Statistics

total_rewards = []

for exp = 1:4
    Random.seed!(exp)
    p = ToEnd(0)
    traj_rewards = sum([step.r for step in stepthrough(m,p,belief_updater, max_steps=100)])
    push!(total_rewards, traj_rewards)
end

@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(5))

Mean Total Reward: -10.750, StdErr Total Reward: 0.224

### Helper Functions

In [26]:
function collisionStatus(s, a)
    e = mdp(m)
    v, om = action
    v = clamp(v, 0.0, e.v_max)
    om = clamp(om, -e.om_max, e.om_max)

    # propagate dynamics without wall considerations
    x, y, th, _ = s
    dt = e.dt

    # dynamics assume robot rotates and then translates
    next_th = wrap_to_pi(th + om*dt)

    # make sure we arent going through a wall
    p0 = SVector(x, y)
    heading = SVector(cos(next_th), sin(next_th))
    des_step = v*dt
    next_x, next_y = AA228FinalProject.legal_translate(e.room, p0, heading, des_step)

    # Determine whether goal state or stairs have been reached
    grn = mdp(m).room.goal_rect
    gwn = mdp(m).room.goal_wall
    srn = mdp(m).room.stair_rect
    swn = mdp(m).room.stair_wall
    gr = mdp(m).room.rectangles[grn]
    sr = mdp(m).room.rectangles[srn]    
    
    next_status = 1.0*AA228FinalProject.contact_wall(gr, gwn, [next_x, next_y]) - 1.0*AA228FinalProject.contact_wall(sr, swn, [next_x, next_y])
    sp = RoombaState(next_x, next_y, next_th, next_status)
    
    return e, gr, gwn, sr, swn, next_x, next_y, next_th, next_status, sp
end

collisionStatus (generic function with 1 method)

In [ ]:
function rew(m::RoombaPOMDP{Bumper,Bool}, s, a, sp)
    # penalty for each timestep elapsed
    cum_reward = mdp(m).time_pen

    # penalty for bumping into wall (not incurred for consecutive contacts)
    previous_wall_contact = wall_contact(m,s)
    current_wall_contact = wall_contact(m,sp)
    if(!previous_wall_contact && current_wall_contact)
        cum_reward += mdp(m).contact_pen
    end

    # terminal rewards
    cum_reward += mdp(m).goal_reward*(sp.status == 1.0)
    cum_reward += mdp(m).stairs_penalty*(sp.status == -1.0)

    return cum_reward  
end